In [1]:
!nvidia-smi

Mon Mar 14 13:56:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [34]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [36]:
training_data = datasets.FashionMNIST(
    root= 'data',
    train= True,
    download= True,
    transform= ToTensor()
)

test_data = datasets.FashionMNIST(
    root= 'data',
    train= False,
    download= True,
    transform= ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size= 10)
test_dataloader = DataLoader(test_data, batch_size= 10)

In [41]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(), #softmax 在 loss function 中的 crossentropyloss
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
  
model = NeuralNetwork().to(device)

In [24]:
learning_rate = 1e-3
batch_size = 10
epochs = 1

In [25]:
loss = nn.CrossEntropyLoss() #有含softmax

In [26]:
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

In [27]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  print(size)
  for batch, (X, y) in enumerate(dataloader): #enumerate 將資料一層層抽出
    
    #compute prediction and loss
    pred = model(X.to(device))
    loss = loss_fn(pred, y.to(device))

   #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 10 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X.to(device))
      test_loss += loss_fn(pred, y.to(device)).item()
      correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f'Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [28]:
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)


for t in range(epochs):
  print(f'Epoch {t+1}\n--------------------')
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
  print('Done!')

Epoch 1
--------------------
60000
loss: 4.861573 [    0/60000]
loss: 4.859564 [  100/60000]
loss: 4.852628 [  200/60000]
loss: 4.837476 [  300/60000]
loss: 4.852942 [  400/60000]
loss: 4.853873 [  500/60000]
loss: 4.846477 [  600/60000]
loss: 4.859248 [  700/60000]
loss: 4.847826 [  800/60000]
loss: 4.840562 [  900/60000]
loss: 4.836454 [ 1000/60000]
loss: 4.848810 [ 1100/60000]
loss: 4.838127 [ 1200/60000]
loss: 4.848467 [ 1300/60000]
loss: 4.857254 [ 1400/60000]
loss: 4.838419 [ 1500/60000]
loss: 4.851539 [ 1600/60000]
loss: 4.840825 [ 1700/60000]
loss: 4.848750 [ 1800/60000]
loss: 4.831192 [ 1900/60000]
loss: 4.846412 [ 2000/60000]
loss: 4.842194 [ 2100/60000]
loss: 4.837165 [ 2200/60000]
loss: 4.816484 [ 2300/60000]
loss: 4.837239 [ 2400/60000]
loss: 4.844429 [ 2500/60000]
loss: 4.817622 [ 2600/60000]
loss: 4.828120 [ 2700/60000]
loss: 4.829695 [ 2800/60000]
loss: 4.800689 [ 2900/60000]
loss: 4.825496 [ 3000/60000]
loss: 4.832018 [ 3100/60000]
loss: 4.808062 [ 3200/60000]
loss: 4.